# Create input from WRTDS

Reads WRTDS discharge. Normalized the discharge and create input file to run model
and creates binary river forcing file

Q : m3/s

TP : mg/L

    L. Gloege (2017-11-02)

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import struct
import sys

In [2]:
def plot_input():
    """ plot location of river to check position """
    import matplotlib.pyplot as plt
    mask = (depth == 0)
    depthMask = np.ma.array(depth, mask=mask)
    plt.pcolor(lon, lat, depthMask)
    plt.plot(lon[lonInd], lat[latInd], 'r.')
    plt.plot(lon[lonInd + 1], lat[latInd], 'r.')
    plt.plot(lon[lonInd + 1], lat[latInd + 1], 'r.')
    plt.plot(lon[lonInd], lat[latInd + 1], 'r.')
    plt.show()

### Load export_binary function

In [3]:
%run -i /Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/scripts/export_binary.py

### Load get_model_grid function

In [4]:
%run -i /Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/scripts/get_model_grid.py

### Get grid and station information

In [5]:
# Get model grid
grid = get_model_grid()

# Store fileName, lat of mouth and lon of mouth
wrtds_dir = '/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/data/raw/wrtds/' 
stations = {'053210': [f'{wrtds_dir}Fox_053210.csv', 44.54, -88.005, 'Fox'],
            '04126010': [f'{wrtds_dir}Manistee_04126010.csv', 44.25, -86.34453, 'Manistee'],
            '04122030': [f'{wrtds_dir}Muskegon_04122030.csv', 43.227089, -86.341987, 'Muskegon'],
            '04102080': [f'{wrtds_dir}St_Joseph_04102080.csv', 42.114167, -86.488333, 'St Joseph'],
            '04119400': [f'{wrtds_dir}Grand_04119400.csv', 43.05835, -86.25088, 'Grand'],
            '04057004': [f'{wrtds_dir}Manistique_04057004.csv', 45.948, -86.246, 'Manistique'],
            '04122500': [f'{wrtds_dir}Pere_Marquette_04122500.csv', 43.951667, -86.460278, 'Pere Marquette'],
            '383088': [f'{wrtds_dir}Menominee_383088.csv', 45.0947, -87.59121, 'Menominee'],
            '04108660': [f'{wrtds_dir}Kalamazoo_04108660.csv', 42.67669, -86.21531, 'Kalamazoo'],
            '413640': [f'{wrtds_dir}Milwaukee_413640.csv', 43.025556, -87.894831, 'Milwaukee'],
            '603095': [f'{wrtds_dir}Sheboygan_603095.csv', 43.748976, -87.702149, 'Sheboygan']}

### Define which river and year you want to run

In [45]:
# Read data
siteID = '053210'
riverName = stations[siteID][3]
year = 2007
df = pd.read_csv(stations[siteID][0], index_col='Date')

# Convert index to dates
df.index = df.index.astype('datetime64[ns]')

# Select dates
mask = (df.index>= str(year) + '-01-01') & (df.index<=str(year+1) + '-01-01')
df = df[mask]

# Integrate under curve and normalize
dx = 86400
Q_ann = np.trapz(df['Q'].values, dx=dx)
df['Q_norm'] = df['Q'] / Q_ann

# Sanity check, this should be 1
test = np.trapz(df['Q_norm'].values, dx=dx) 

# Reorder columns
df = df[['Q', 'Q_norm', 'TP']]


# River
latMouth = stations[siteID][1]
lonMouth = stations[siteID][2]

# Find river mouth in grid (the comma unpacks the tuple)
lonInd, = np.where((grid['lon'] < lonMouth + grid['dlon']) & \
                   (grid['lon'] > lonMouth - grid['dlon']))
latInd, = np.where((grid['lat'] < latMouth + grid['dlat']) & \
                   (grid['lat'] > latMouth - grid['dlat']))

# Fine tune the river mouth location
if (riverName == 'Fox'):
    lonInd = lonInd + 1
    latInd = latInd + 1
if (riverName == 'Manistee'):
    latInd = latInd + 1
if (riverName == 'Muskegon'):
    lonInd = lonInd - 4
if (riverName == 'Grand'):
    lonInd = lonInd - 2
if (riverName == 'Manistique'):
    latInd = latInd - 2
if (riverName == 'Pere Marquette'):
    lonInd = lonInd - 3
if (riverName == 'Kalamazoo'):
    lonInd = lonInd - 2
if (riverName == 'Menominee'):
    lonInd = lonInd - 1
if (riverName == 'Sheboygan'):
    latInd = latInd - 1

### Calculate flux

In [46]:
# add singleton dimension to flux
#flux = np.expand_dims(flux, axis=1) / volCell[latInd, lonInd]
Q = np.expand_dims(df['Q_norm'].values, axis=1)
flux = Q * (grid['volLake'] / grid['volCell'][latInd, lonInd]) # arbs/s

# Initialize tracer grid, zero matrix [366 1 276 200]
nt, nz = flux.shape
ny, = grid['lat'].shape
nx, = grid['lon'].shape
surfaceForcing = np.zeros((nt, nz, ny, nx), dtype='f')

# add flux (THIS IS WHAT YOU WANT TO BE BINARY)
surfaceForcing[:, 0, latInd, lonInd] = flux.reshape(-1,1)    

print(riverName, ' : ', siteID)
print('')
print('latInd : {0} lonInd : {1}'.format(latInd,lonInd))
print('surfaceForcing shape: {0}'.format(surfaceForcing.shape))
print('flux shape: {0}'.format(flux.shape))
print('sum : {0}'.format(surfaceForcing.sum()))
print('This should always be 1 : ', test)


Sheboygan  :  603095

latInd : [130] lonInd : [23]
surfaceForcing shape: (366, 1, 276, 200)
flux shape: (366, 1)
sum : 4.584115028381348
This should always be 1 :  1.0


### Save binary

In [47]:
# Write to binary file
#fileName = '/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/model/inputs/river_' + siteID + '_'+ str(year) + '.bin'
dir_model = '/Users/gloege/Documents/Projects/MITgcm-Michigan-Phosphorus/model/inputs/'
fileName = f'{dir_model}inputs/river_{siteID}_{year}.bin'
export_binary(fileName, surfaceForcing)

# Display output file
print('output file: ' + fileName)

output file: /Users/gloege/Documents/Projects/lakeMichigan/model/inputs/tracer/river_603095_2007.bin


In [8]:
for siteID in stations.keys():
    print(siteID)

053210
04126010
04122030
04102080
04119400
04057004
04122500
383088
04108660
413640
603095
